Read Parquet file

In [1]:
df = spark.read.parquet("Deepu/names.parquet")

In [2]:
df.count()

In [3]:
df.printSchema()

In [4]:
firstNamesDF = df.select("firstName", "year")
firstNamesDF.show()

In [5]:
firstNamesDF.select("firstName").distinct().count()

Top 5 Popular Name in 1980

In [6]:
popular = (df.filter(df['year'] == 1980).
           filter(df['gender'] == 'F').
           orderBy(df['total'].desc(), df['firstName']).
           select("firstName").
           limit(5))

Convert into lowercase

In [7]:
from pyspark.sql.functions import udf
lower = udf(lambda s:s.lower())

Table of 2010

In [8]:
ssn2010 = df.filter(df['year'] == 2010).\
             select(df['total'].alias("total2010"),
                    df['gender'].alias("gender2010"),
                    df['firstName'].alias("firstName2010"),
                    lower(df['firstName']).alias("name2010"))

Table 1930

In [9]:
ssn1930 = df.filter(df['year'] == 1930).\
             select(df['total'].alias("total1930"),   
                    df['gender'].alias("gender1930"),
                    df['firstName'].alias("firstName1930"), 
                    lower(df['firstName'].alias("name1930"))

Join table 2010 & 1930

In [10]:
joined = ssn2010.filter(ssn2010.gender2010 == "F").\
                 join(ssn1930.filter(ssn1930.gender1930 == "F"), ssn2010.name2010 == ssn1930.name1930).\
                 orderBy(ssn2010.total2010.desc()).\
                 limit(10).\
                 select(ssn2010.firstName2010.alias("name"), ssn1930.total1930, ssn2010.total2010)

To read xml files write the below statement in the shell

In [11]:
pyspark2 --master local --packages com.databricks:spark-xml_2.10:0.4.1

Read XML (not for all xml it will have changes according to references)

In [12]:
employees_df = spark.read.format("com.databricks.spark.xml").option("inferSchema", "true").option("rootTag", "employees").option("rowTag", "employee").load("datasets/employees.xml")

Read from RDBMS write the below statement in the shell

In [13]:
pyspark2 --master local --driver-class-path mysql-connector-java-5.1.46.jar --jars mysql-connector-java-5.1.46.jar

In [14]:
jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://mysqldb.projectpro.com:5147") \
    .option("dbtable", "retail_db.order_items") \
    .option("user", "labuser") \
    .option("password", "DeepuPillai") \
    .load()